# CreditOne Classification 
Class 5 Task 3

Digging into SVM KNN RF

#### Import modules

In [1]:
#enable auto complete
%config IPCompleter.greedy=True

In [2]:
#imports
#numpy,pandas,scipy,math,matplotlib
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

#estimators
#svm
from sklearn import svm
#knn
from sklearn.neighbors import KNeighborsClassifier
#RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#model metrics
from sklearn.model_selection import cross_val_score
import sklearn.metrics as skm

#cross validation
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import VarianceThreshold


#### import data

In [3]:
#data
rawData = pd.read_csv('default of credit card clients.csv', header=1)
rawData.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,4810,500000,1,3,3,53,0,0,0,0,...,125218,126650,114473,12602,11000,4711,6000,11000,10000,0
1,15096,500000,1,1,3,38,-2,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,1
2,81,470000,2,3,3,33,0,0,0,0,...,69923,29271,29889,6400,7566,3000,960,1000,3000,0
3,14889,450000,2,1,3,37,1,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,0
4,11072,450000,1,1,3,37,-2,-2,-2,-2,...,317,1614,324,303,5041,1,1622,325,316,0


In [4]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null int64
BILL_AMT2                     30000 non-null int64
BILL_AMT3                     30000 non-null int64
BILL_AMT4                     30000 non-null int64
BILL_AMT5                     30000 non-null int64
BILL_AMT6               

In [5]:
df = rawData

In [6]:
#change to categorical

df.SEX = df.SEX.astype('category')
df.MARRIAGE = df.MARRIAGE.astype('category')
df.AGE = df.AGE.astype('category')
df.EDUCATION = df.EDUCATION.astype('category')

In [18]:
#bin Age data
df['AGE'] = pd.cut(df['AGE'], [20,30,40,50,60,70,80],labels=['20-30','30-40','40-50', '50-60', '60-70','80-70'])

In [64]:
#dummiefy age
AGE_dummies = pd.get_dummies(df['AGE'],prefix = "AGE", columns=None, sparse=False, drop_first=False)

In [66]:
AGE_dummies

,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70
0,0,0,0,1,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
5,0,1,0,0,0,0
6,0,0,0,1,0,0
7,0,0,1,0,0,0
8,0,1,0,0,0,0
9,0,1,0,0,0,0


In [ ]:
AGE_dummies

In [67]:
#add age dummies to df and delete age
df=df.join(AGE_dummies)
del df['AGE']

In [73]:
#move default column to end

#make an array of column names
cols = df.columns.tolist()

In [76]:
cols

['ID',
 'LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'PAY_6',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default payment next month',
 'AGE_20-30',
 'AGE_30-40',
 'AGE_40-50',
 'AGE_50-60',
 'AGE_60-70',
 'AGE_80-70']

In [83]:
#move default to the end
cols.insert(20, cols.pop(cols.index('default payment next month')))
cols
df=df[cols]

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null category
EDUCATION                     30000 non-null category
MARRIAGE                      30000 non-null category
PAY_6                         30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                      30000 non-null int64
PAY_AMT2                      30000 non-null int64
PAY_AMT3                      30000 non-null int64
PAY_AMT4                      30000 non-null int64
PAY_AMT5                      30000 non-null int64
PAY_AMT6                      30000 non-null int64
AGE_20-30                     30000 non-null uint8
AGE_30-40                     30000 non-null uint8
AGE_40-50                     30000 non-null uint8
AGE_50-60                     30000 non-null uint8
AGE_60-70      

In [86]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,PAY_6,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70,default payment next month
0,4810,500000,1,3,3,0,114473,12602,11000,4711,6000,11000,10000,0,0,0,1,0,0,0
1,15096,500000,1,1,3,-2,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,81,470000,2,3,3,0,29889,6400,7566,3000,960,1000,3000,0,1,0,0,0,0,0
3,14889,450000,2,1,3,-2,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,11072,450000,1,1,3,-2,324,303,5041,1,1622,325,316,0,1,0,0,0,0,0


In [87]:
df.shape

(30000, 20)

### Select Features


#### Remove low variance features

In [88]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df)

array([[  4.81000000e+03,   5.00000000e+05,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.50960000e+04,   5.00000000e+05,   1.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  8.10000000e+01,   4.70000000e+05,   2.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  1.65820000e+04,   2.00000000e+04,   1.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.15600000e+04,   2.00000000e+04,   2.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  6.39000000e+03,   1.00000000e+04,   1.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [89]:
df.shape

(30000, 20)

## Correlation Matrix

In [ ]:
c = df.corr().abs()

In [93]:
s = c.unstack()
so = s.sort_values(kind="quicksort")

print (so)

#corrMat = df.corr().abs
#print(corrMat)

AGE_80-70                   PAY_AMT4                      0.000081
PAY_AMT4                    AGE_80-70                     0.000081
PAY_AMT5                    AGE_80-70                     0.000103
AGE_80-70                   PAY_AMT5                      0.000103
                            PAY_AMT3                      0.000422
PAY_AMT3                    AGE_80-70                     0.000422
AGE_80-70                   PAY_6                         0.000475
PAY_6                       AGE_80-70                     0.000475
PAY_AMT5                    ID                            0.000652
ID                          PAY_AMT5                      0.000652
                            AGE_50-60                     0.000923
AGE_50-60                   ID                            0.000923
PAY_AMT1                    PAY_6                         0.001496
PAY_6                       PAY_AMT1                      0.001496
PAY_AMT2                    AGE_80-70                     0.00

In [43]:
del df['BILL_AMT1']
del df['BILL_AMT2']
del df['BILL_AMT3']
del df['BILL_AMT4']
del df['BILL_AMT5']


In [45]:
del df['PAY_0']
del df['PAY_2']
del df['PAY_3']
del df['PAY_4']
del df['PAY_5']


In [94]:
df.shape


(30000, 20)

In [95]:
#features
features = df.iloc[:,1:19]
print('Summary of feature sample')
features.head()

Summary of feature sample


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,PAY_6,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,AGE_20-30,AGE_30-40,AGE_40-50,AGE_50-60,AGE_60-70,AGE_80-70
0,500000,1,3,3,0,114473,12602,11000,4711,6000,11000,10000,0,0,0,1,0,0
1,500000,1,1,3,-2,0,0,0,0,0,0,0,0,1,0,0,0,0
2,470000,2,3,3,0,29889,6400,7566,3000,960,1000,3000,0,1,0,0,0,0
3,450000,2,1,3,-2,0,0,0,0,0,0,0,0,1,0,0,0,0
4,450000,1,1,3,-2,324,303,5041,1,1622,325,316,0,1,0,0,0,0


#### Select Dependent Variable

In [96]:
#dependent variable
depVar = df['default payment next month']

In [97]:
depVar.describe()

count    30000.000000
mean         0.221200
std          0.415062
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default payment next month, dtype: float64

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
ID                            30000 non-null int64
LIMIT_BAL                     30000 non-null int64
SEX                           30000 non-null category
EDUCATION                     30000 non-null category
MARRIAGE                      30000 non-null category
PAY_6                         30000 non-null int64
BILL_AMT6                     30000 non-null int64
PAY_AMT1                      30000 non-null int64
PAY_AMT2                      30000 non-null int64
PAY_AMT3                      30000 non-null int64
PAY_AMT4                      30000 non-null int64
PAY_AMT5                      30000 non-null int64
PAY_AMT6                      30000 non-null int64
AGE_20-30                     30000 non-null uint8
AGE_30-40                     30000 non-null uint8
AGE_40-50                     30000 non-null uint8
AGE_50-60                     30000 non-null uint8
AGE_60-70      

### Establish Training Data


In [99]:
#run split
X_train, X_test, y_train, y_test = train_test_split(features, depVar, test_size = 0.30, random_state =123)
print("X_train",X_train.shape,'\n',"X_test", X_test.shape,'\n',"y_train", y_train.shape,'\n',"y_test", y_test.shape)

X_train (21000, 18) 
 X_test (9000, 18) 
 y_train (21000,) 
 y_test (9000,)


# SVM
http://scikit-learn.org/stable/modules/svm.html#classification

SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto’, coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)

In [104]:
SVMclf = svm.SVC()

In [105]:
SVMclf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [106]:
SVMclf_pred = SVMclf.predict(X_test)

In [107]:
print(" ***Support Vector Machine***","\n","accuracy",skm.accuracy_score(y_test, SVMclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, SVMclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, SVMclf_pred),'\n','Precission Score',skm.precision_score(y_test,SVMclf_pred))

 ***Support Vector Machine*** 
 accuracy 0.779333333333 
 average_precision_score 0.232245497368 
 Kappa 0.0296021824993 
 Precission Score 0.626666666667


# SVM **TUNED**
http://scikit-learn.org/stable/modules/svm.html#classification

SVC(C=1.0, kernel=’rbf’, degree=3, gamma=’auto’, coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape=’ovr’, random_state=None)

In [134]:
from sklearn.model_selection import GridSearchCV

In [135]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

In [136]:
svc = svm.SVC()

In [ ]:
SVM_clf = GridSearchCV(svc, parameters)

In [ ]:
SVM_clf.fit(X_train, y_train)

In [ ]:
sorted(SVM_clf.cv_results_.keys())

In [ ]:
SVM_clf_pred = SVM_clf.predict(X_test)

In [ ]:
print(" ***Support Vector Machine TUNED***","\n","accuracy",skm.accuracy_score(y_test, SVM_clf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, SVM_clf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, SVM_clf_pred),'\n','Precission Score',skm.precision_score(y_test,SVM_clf_pred))

In [ ]:
confusion_SVM = skm.confusion_matrix(y_test, SVMclf_pred)
print(confusion_LR)

# 0: negative class (doesn't have default status)
# 1: positive class (has default status)


#        Pr_0  Pr_1
# Act_0   TN    FP 
# Act_1   FN    TP

# K-Nearest Neighbor Classifier

http://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=1, **kwargs)

In [108]:
KNNclf = KNeighborsClassifier(n_neighbors=30)

In [109]:
KNNclf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=30, p=2,
           weights='uniform')

In [110]:
KNNclf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': 1,
 'n_neighbors': 30,
 'p': 2,
 'weights': 'uniform'}

In [111]:
KNNclf_pred = KNNclf.predict(X_test)

In [112]:
print(" ***K-Nearest Neighbor Classifier***","\n","accuracy",skm.accuracy_score(y_test, KNNclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, KNNclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, KNNclf_pred),'\n','Precision Score',skm.precision_score(y_test,KNNclf_pred))

 ***K-Nearest Neighbor Classifier*** 
 accuracy 0.777222222222 
 average_precision_score 0.233285951787 
 Kappa 0.0403372141752 
 Precision Score 0.5


# K-Nearest Neighbor Classifier **TUNED**

http://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=1, **kwargs)

In [113]:
from sklearn.model_selection import GridSearchCV

In [114]:
parameters = {'n_neighbors':[5,10,15,20,25,30,35,40,45,50], 'leaf_size':[5,10,15,20,25,30,35,40,45,50]}

In [115]:
KNNT = KNeighborsClassifier()

In [116]:
KNNTclf = GridSearchCV(KNNT, parameters, cv = 10, scoring = 'accuracy')

In [117]:
KNNclf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=30, p=2,
           weights='uniform')

In [118]:
KNNclf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': 1,
 'n_neighbors': 30,
 'p': 2,
 'weights': 'uniform'}

In [119]:
KNNclf_pred = KNNclf.predict(X_test)

In [120]:
print(" ***K-Nearest Neighbor Classifier***","\n","accuracy",skm.accuracy_score(y_test, KNNclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, KNNclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, KNNclf_pred),'\n','Precision Score',skm.precision_score(y_test,KNNclf_pred))

 ***K-Nearest Neighbor Classifier*** 
 accuracy 0.777222222222 
 average_precision_score 0.233285951787 
 Kappa 0.0403372141752 
 Precision Score 0.5


# Random Forest

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)

In [121]:
RFclf = RandomForestClassifier(max_depth=2, random_state=0)

In [122]:
RFclf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [123]:
RFclf_pred = RFclf.predict(X_test)

In [124]:
print(" ***Random Forest***" ,'\n',"accuracy",skm.accuracy_score(y_test, RFclf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, RFclf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, RFclf_pred),'\n','Precision Score',skm.precision_score(y_test,RFclf_pred))

 ***Random Forest*** 
 accuracy 0.777222222222 
 average_precision_score 0.222777777778 
 Kappa 0.0 
 Precision Score 0.0


/Users/degraff/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [125]:
print(RFclf.feature_importances_)

[ 0.1403961   0.01524298  0.          0.          0.34405953  0.02824097
  0.06937427  0.08816473  0.14560334  0.07755607  0.0719232   0.01943882
  0.          0.          0.          0.          0.          0.        ]


# Linear SVC
http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

LinearSVC(penalty=’l2’, loss=’squared_hinge’, dual=True, tol=0.0001, C=1.0, multi_class=’ovr’, fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)

In [126]:
lin_clf = svm.LinearSVC()

In [127]:
lin_clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [128]:
lin_clf_pred = lin_clf.predict(X_test)

In [129]:
print(" ***Linear SVC***" ,'\n',"accuracy",skm.accuracy_score(y_test, lin_clf_pred),'\n',"average_precision_score",skm.average_precision_score(y_test, lin_clf_pred),'\n',"Kappa",skm.cohen_kappa_score(y_test, lin_clf_pred),'\n','Precision Score',skm.precision_score(y_test,lin_clf_pred))

 ***Linear SVC*** 
 accuracy 0.258333333333 
 average_precision_score 0.2189449576 
 Kappa -0.0106483559447 
 Precision Score 0.218539054966


# Accuracy

In [130]:
SVMscores = cross_val_score(SVMclf, X_train, y_train, cv=15)
RFscores = cross_val_score(RFclf, X_train, y_train, cv=15)
KNNscores = cross_val_score(KNNclf, X_train, y_train, cv=15)

In [131]:
LINscores = cross_val_score(lin_clf, X_train, y_train, cv=15)

In [132]:
print(" SVM Accuracy: %0.2f (+/- %0.2f)" % (SVMscores.mean(), SVMscores.std() * 2),
    '\n',"KNN Accuracy: %0.2f (+/- %0.2f)" % (KNNscores.mean(), KNNscores.std() * 2),
    '\n', "RF Accuracy: %0.2f (+/- %0.2f)" % (RFscores.mean(), RFscores.std() * 2),
    '\n',"LinSVC Accuracy: %0.2f (+/- %0.2f)" % (LINscores.mean(), LINscores.std() * 2))

 SVM Accuracy: 0.78 (+/- 0.01) 
 KNN Accuracy: 0.78 (+/- 0.01) 
 RF Accuracy: 0.78 (+/- 0.00) 
 LinSVC Accuracy: 0.69 (+/- 0.25)


In [133]:
print(" LINscores","\n",LINscores,"\n",
"SVMscores","\n",
      SVMscores,"\n",
"SRscores","\n",
      RFscores,"\n",
"KNNscores","\n",
      KNNscores)

 LINscores 
 [ 0.76659529  0.77944325  0.40970735  0.50392577  0.775       0.76285714
  0.77571429  0.77928571  0.58714286  0.51142857  0.77928571  0.77197999
  0.77984274  0.65117941  0.77197999] 
 SVMscores 
 [ 0.78229836  0.78229836  0.78087081  0.78015703  0.78357143  0.77714286
  0.77928571  0.78        0.78071429  0.78142857  0.78214286  0.78127234
  0.77340958  0.77984274  0.78413152] 
 SRscores 
 [ 0.77944325  0.77944325  0.77944325  0.77944325  0.77928571  0.77928571
  0.77928571  0.77928571  0.77928571  0.77928571  0.77928571  0.77984274
  0.77984274  0.77984274  0.77984274] 
 KNNscores 
 [ 0.78158458  0.78372591  0.7780157   0.77872948  0.78142857  0.78142857
  0.77642857  0.77857143  0.77857143  0.78285714  0.78428571  0.77626876
  0.77912795  0.77769836  0.77769836]


## Making Predictions

# Tuning